# Śledzenie obiektów

## Wstęp
W erze cyfrowej, w obliczu rosnącej lawinowo ilości danych wideo, zdolność do ich automatycznego rozpoznawania i interpretowania staje się kluczowa w wielu dziedzinach – od bezpieczeństwa publicznego po autonomiczne pojazdy. Technologie oparte na głębokim uczeniu rewolucjonizują sposób, w jaki przetwarzamy informacje wizualne. Kluczowym wyzwaniem jest tu detekcja i śledzenie obiektów na filmach wideo.

Celem tego zadania jest opracowanie algorytmu, który będzie w stanie analizować sekwencje ruchów w grze "trzy kubki". Uczestnicy mają za zadanie określić końcową pozycję kubków po serii ruchów, korzystając z analizy statycznych obrazów z każdej klatki nagrania.

In [ ]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

# Poniższe funkcje ułatwiają pracę z dostarczonymi danymi
# W kolejnych komórkach zobaczysz przykłady ich użycia
from utils.utils import get_level_info, get_video_data, display_video, download_and_replace_data

FINAL_EVALUATION_MODE = False
# W czasie sprawdzania Twojego rozwiązania, zmienimy tę wartość na True
# Wartość tej flagi M U S I zostać ustawiona na False w rozwiązaniu, które nam nadeślesz!

images, coordinates, target, path_to_images = get_video_data(level=2,video_id=0,dataset="example")
display_video(images,rescale=0.7,FINAL_EVALUATION_MODE=FINAL_EVALUATION_MODE)

## Zadanie 2: Okluzje, rozmycia, przesłonienie

Celem zadania jest opracowanie algorytmu, który potrafi przetwarzać sekwencje obrazów z gry w trzy kubki, nawet gdy występują rozmycia czy przesłonięcia. Zadanie ma na celu nauczenie maszyny wykorzystywania ciągłości informacji z kolejnych klatek, aby mimo chwilowych utrudnień w percepcji, mogła skutecznie określić końcową pozycję kubków.

Napisz algorytm który poradzi sobie z trudnieszym zbiorem danych - `Level_2`. Składa się on z animacji, w których pojawiają się dodatkowe utrudnienia:
- Przesłonięcia obiektu przez inny, powodujące, że są one traktowane jako jeden,
- Prostokąty ograniczające nie są już idealnie dopasowane do obiektów,
- Prostokąty ograniczające nie są widoczne we wszystkich klatkach.

Będziesz miał dostęp zarówno do wszystkich klatek animacji, jak i do oznaczonych przez nas prostokątów ograniczających, w których znajdują się kubki. Co ważne, algorytm, który będziesz tworzył ma korzystać jedynie z informacji o prostokątach ograniczających. W tym zadaniu, klatki wideo są dostarczone jedynie do wizualizacji przykładów i algorytmu, na własne potrzeby.

Punkty za to zadanie będą przyznane za osiągnięcie jak najdokładniejszych predykcji na zbiorze testowym. Kryterium będzie *accuracy* i spodziewamy się wyników powyżej `80%`. Ewaluacja na zbiorze testowym będzie dokonana przez organizatorów.

## Pliki zgłoszeniowe
Tylko ten notebook zawierający **kod** oraz **krótki raport** opisujący Twoje rozwiązanie (do 300 słów). Miejsce na raport znajdziesz na końcu tego notebooka.

## Ograniczenia
- Twoja funkcja powinna zwracać predykcje w maksymalnie 5 minut używając Google Colab bez GPU.

## Uwagi i wskazówki
- Testuj swoje rozwiązanie na zbiorze plików wideo `level_2`.
- **Skuteczność modelu**: przetestuj skuteczność modelu na zbiorze walidacyjnym używając dostarczonej przez nas funkcji **submission_script**, umieść ten wynik w raporcie.

## Ewaluacja
Pamiętaj, że podczas sprawdzania flaga `FINAL_EVALUATION_MODE` zostanie ustawiona na `True`. Za pomocą skryptu `validation_script.py` możesz upewnić się, że Twoje rozwiązanie zostanie prawidłowo wykonane na naszych serwerach oceniających.

Za to podzadanie możesz zdobyć pomiędzy 0 i 0.5 punktów. Zdobędziesz 0 punktów jeśli Twoje accuracy na zbiorze testowym będzie poniżej 50%. Jeśli będzie większe niż 95%, otrzymasz 0.5 punktu. Pomiędzy tymi wartościami, wynik rośnie liniowo z wartością metryki.

# Kod startowy

In [3]:
# Poniższe biblioteki są wystarczające do wykonania wszystkich zadań
# Jeśli jednak chcesz użyć innych, sprawdź czy są dostępne na serwerze (requirements.txt)
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import IPython.display
import json
import PIL
import sklearn as sk

In [ ]:
# funkcja pomocnicza do ładowania danych
images, _, _, _ = get_video_data(level=2,video_id=0,dataset="example")

with open(os.path.join(os.getcwd(),'example_tracks','tracks_2_0.json'), 'r') as f:
    tracks = json.load(f)

for key in tracks.keys():
    tracks[key] = [tuple(el) for el in tracks[key]]

# funkcja pomocnicza do wyświetlania danych
display_video(images,
                tracks=tracks,
                rescale=0.7,
                FINAL_EVALUATION_MODE=FINAL_EVALUATION_MODE)

In [ ]:
# Pobieranie danych do podzadań 1, 2 i 3 (około ~646Mb), skrypt będzie wykonywał się parę minut
# Wystarczy, że pobierzesz dane tylko raz. Na serwerze sprawdzającym dane będą już pobrane,
# struktura plików będzie identyczna jak tutaj
if not FINAL_EVALUATION_MODE:
    download_and_replace_data()

In [6]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

# funkcja pomocnicza do testowania algorytmu
def submission_script(algorithm,level,verbose=False,dataset="valid"):
    num_videos, _ = get_level_info(level=level,dataset=dataset)
    correct = []
    exception_messages = set()
    for video_number in range(num_videos):
        _, coordinates, target, _ = get_video_data(level=level,video_id=video_number,dataset=dataset)
        try:
            prediction = algorithm(coordinates)
            if tuple(target) == tuple(prediction):
                correct.append(1)
            else:
                correct.append(0)
            if verbose:
                print(f"Video: animation_{str(video_number).zfill(4)}")
                print(f"Prediction: {prediction}")
                print(f"Target:     {target}")
                print(f"Score: {tuple(target) == tuple(prediction)}", end='\n\n')
        except Exception as e:
            correct.append(0)
            exception_messages.add(str(e))
    if verbose:
        print(f"Accuracy: {np.mean(correct)}")
        print(f"Correctness: {correct}")
    return np.sum(correct) / num_videos, correct, exception_messages

# Twoje rozwiązanie

In [ ]:
# Hubert Jastrzębski - V LO Kraków

from itertools import permutations 
from statistics import median
from copy import deepcopy
import math

np.random.seed(42)
torch.manual_seed(42)

In [14]:
def getPerspectiveTransform(sourcePoints, destinationPoints):
    a = np.zeros((8, 8))
    b = np.zeros((8))
    for i in range(4):
        a[i][0] = a[i+4][3] = sourcePoints[i][0]
        a[i][1] = a[i+4][4] = sourcePoints[i][1]
        a[i][2] = a[i+4][5] = 1
        a[i][3] = a[i][4] = a[i][5] = 0
        a[i+4][0] = a[i+4][1] = a[i+4][2] = 0
        a[i][6] = -sourcePoints[i][0]*destinationPoints[i][0]
        a[i][7] = -sourcePoints[i][1]*destinationPoints[i][0]
        a[i+4][6] = -sourcePoints[i][0]*destinationPoints[i][1]
        a[i+4][7] = -sourcePoints[i][1]*destinationPoints[i][1]
        b[i] = destinationPoints[i][0]
        b[i+4] = destinationPoints[i][1]

    x = np.linalg.solve(a, b)
    x.resize((9,), refcheck=False)
    x[8] = 1
    return x.reshape((3,3))

original_points = np.float32([[320, 240], [150, 130], [320, 85], [490, 130]])

top_view_points = np.float32([[0, 0], [0, 1000], [1000, 1000], [1000, 0]])
matrix = getPerspectiveTransform(original_points, top_view_points)

def transform_point(p):
  px = (matrix[0][0]*p[0] + matrix[0][1]*p[1] + matrix[0][2]) / ((matrix[2][0]*p[0] + matrix[2][1]*p[1] + matrix[2][2]))
  py = (matrix[1][0]*p[0] + matrix[1][1]*p[1] + matrix[1][2]) / ((matrix[2][0]*p[0] + matrix[2][1]*p[1] + matrix[2][2]))
  return (px, py)


In [15]:
def distance(a, b):
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def your_algorithm_task_2(coordinates): # nie zmieniaj nazwy funkcji
    points = [[((xa + xb) / 2, (ya + yb) / 2) for (xa, ya, xb, yb) in coordinates[name]] for name in coordinates]
    (b1, res1) = subtask(points)

    new_points = []
    for i in range(len(points)) :
        new_points.append(points[len(points)-i-1])

    (b2, res2) = subtask(new_points)

    if (b1 < b2 * 1.008): 
        return res1
    return res2

def subtask(points): 
    points = [[transform_point(p) for p in ps] for ps in points] 
    points = [[(round(q[0], 1), round(q[1], 1)) for q in p] for p in points]

    points[0].sort()
    velocity = [[0, 0, 0]]
    max_velocity = 0
    
    sum_best_p = 0
    best_res = 0
    for i in range(1, len(points)):
        p = deepcopy(points[i])
        best_res = 0
        if len(p) == 0:
            points[i] = deepcopy(points[i - 1])
        
        elif len(p) == 1:
            best_res = math.inf
            best_j = 0
            for j in [0, 1, 2]:
                points[i] = deepcopy(points[i - 1])
                points[i][j] = deepcopy(p[0])
                res = (distance(points[i][j], points[i - 1][j]))

                if res < best_res:
                    best_res = res 
                    best_j = j

            points[i] = deepcopy(points[i - 1])
            points[i][best_j] = deepcopy(p[0])


        elif len(p) == 2:
            best_res = math.inf
            best_p = deepcopy(points[i - 1])
            for perm in permutations([0, 1, 2], 2):
                points[i] = deepcopy(points[i - 1])
                points[i][perm[0]] = deepcopy(p[0])
                points[i][perm[1]] = deepcopy(p[1])
                r = [(distance(points[i][j], points[i - 1][j])) for j in [0, 1, 2]]
                mv = min(velocity[i-1])
                res = sum(r)

                if res < best_res:
                    best_res = res 
                    best_p = deepcopy(points[i])

            points[i] = best_p

        elif len(p) == 3:
            best_res = math.inf
            best_p = deepcopy(points[i - 1])
            for perm in permutations([0, 1, 2]):
                points[i] = [p[j] for j in perm]
                r = [(distance(points[i][j], points[i - 1][j])) for j in [0, 1, 2]]

                res = sum(r)
                
                if res < best_res:
                    best_res = res 
                    best_p = deepcopy(points[i])
            points[i] = best_p
            
        if i > 0 :
            v = [distance(points[i][j], points[i-min(i,3)][j])/min(i,3) for j in range(3)]
            velocity.append(v)
            max_velocity = max(max_velocity, max(velocity[i]))
        sum_best_p += best_res
        
    rp = [points[-1][i][0] for i in [0, 1, 2]]
    res = [rp.index(i) for i in [min(rp), median(rp), max(rp)]]

    return (sum_best_p, res)

In [ ]:
# Sprawdź jak działa Twój algorytm
if not FINAL_EVALUATION_MODE:
    accuracy, correctness, _ = submission_script(
        algorithm=your_algorithm_task_2,
        level=2,
        verbose=True,
        dataset="train")

In [11]:
# zapisz swój raport do zmiennej poniżej, abyśmy mogli go później automatycznie odczytać sprawdzaczką
raport_2 = \
"""
Raport z zadania:
zamieniam współrzędne prostokątów na ich środki
transformuję te punkty na takie trochę bardziej z góry
rozpatruję po kolei klatki
dla każdej klatki patrzę która permutacja środków daje najmniejszą sumę odległości
(jeśli niektóre środki zniknęły, to zostawiam je takie same jak w poprzedniej klatce)
"""